# I. Lý thuyết

1) Mạng tích chập CNN sẽ được thực hiện như thế nào?

A. Phân chia feature map thành các ma trận theo chiều channel và thực hiện tích chập giữa mỗi ma trận kernel với từng chiều.

B. Thực hiện phép nhân ma trận kernel với chuyển vị của ma trận local receptive field để thu được giá trị feature map output.

C. Tính trung bình các phân tử trên một ma trận local receptive field.

D. Di chuyển vùng local receptive field theo chiều từ trái sang phải và từ trên xuống dưới và nhân tích chập với kernel để thu được ma trận feature map.


Đáp án: D


2) Thực hiện phép nhân tích chập giữa ma trận đầu vào kích thước width và height lần lượt là $(W, H)$ với bộ lọc kích thước $(F, F)$ và bước nhảy $S$ thì thu được ma trận đầu ra với kích thước width và height $(W', H')$ là bao nhiêu? Hãy lựa chọn công thức tổng quát nhất.

A. $W' =⌊ \frac{W-F}{S} ⌋ + 1;~ H' =⌊ \frac{H-F}{S} ⌋ + 1$

B. $W' =\lceil \frac{W-F}{S} \rceil + 1;~ H' = \lceil \frac{H-F}{S} \rceil + 1$ 

C. $W' =\lfloor \frac{W-F+2P}{S} \rfloor + 1;~ H' = \lfloor \frac{H-F+2P}{S} \rfloor + 1$ với $P$ là số lượng các véc tơ 0 padding ở bên ngoài đều hai phía.

D. $W' =\lceil \frac{W-F+P}{S} \rceil + 1;~ H' = \lceil \frac{H-F+P}{S} \rceil + 1$ với $P$ là số lượng các véc tơ 0 padding ở bên ngoài đều hai phía.

Đáp án: C

3) Mạng CNN sẽ học được những gì qua các layers?

A. Tại những layers đầu mỗi channels sẽ giúp phân biệt một loại đặc trưng, của vật thể. Những đặc trưng này sau đó được trải phẳng và đưa vào mạng MLP để phân loại vật thể.

B. Những layers cuối cùng sẽ zoom vào từng chi tiết của vật thể để phân loại vật thể.

C. Tại những layers đầu mạng sẽ học được các đặc trưng chung như các edge dọc, ngang, chéo,.... Các đặc trưng chi tiết và tổng quát giúp nhận diện vật thể được học tại những layers cuối cùng.

D. Các layers đầu tiên sẽ tập trung vào các chi tiết bộ phận của vật thể.

Đáp án: C


4) Kiến trúc chung của một mạng CNN là gì?

A. _[Conv -> BatchNorm -> Activation -> Maxpooling] x n_

B. _[BatchNorm -> Conv -> Activation -> Maxpooling] x n_ 

C. _[Conv -> BatchNorm -> Maxpooling -> Activation -> Maxpooling] x n_ 

D. _[Conv -> BatchNorm -> Activation -> Maxpooling] x n -> Flatten -> [FullyConnected -> Activation] x m -> Softmax_ 

Đáp án: D

5) Thông thường đặc điểm kích thước feature map của mạng neural sẽ như thế nào ?

A. Kích thước feature map tăng dần gấp đôi qua thời gian.

B. Kích thước feature map sẽ duy trì không đổi qua thời gian.

C. Kích thước mạng sẽ giảm dần gấp đôi sau mỗi một lần downsampling và sau đó tăng dần gấp đôi sau mỗi lần upsampling để khôi phục về kích thước ảnh input.

D. Kích thước mạng thường giảm gấp đôi sau một lần downsampling và số lượng các filters tăng dần.

Đáp án: D

# II. Thực hành

6) Thực hiện xây dựng và huấn luyện một mạng CNN ngẫu nhiên trên bộ dữ liệu [Dog and Cat](https://www.kaggle.com/c/dog-vs-cat-classification/data). Lưu ý cần thực hiện [chuẩn hóa dữ liệu](https://github.com/pytorch/examples/issues/112) đối với bộ dữ liệu ImageNet trước khi huấn luyện.

7) Grid Search kiến trúc CNN dựa trên việc tổng quát hóa một thiết kế câu 6.

8) Huấn luyện lại mô hình bằng kiến trúc ResNet và MobileNet. Đánh giá chi phí tính toán và submit kết quả độ chính xác trên các ảnh thuộc folder test.

9) Thử nghiệm các phương pháp Augmentation như Flip, Rotation, Random Crop, Bright Contrast để cải thiện kết quả mô hình.

10) Lập bảng kết quả các thử nghiệm đã sử dụng.

In [6]:
from google.colab import drive
import os
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [7]:
%cd /content/gdrive/MyDrive/Colab Notebooks/DeepLearning2/5.CNN

/content/gdrive/MyDrive/Colab Notebooks/DeepLearning2/5.CNN


In [ ]:
# !unzip ../data/dogvscat/train.zip -d ../data/dogvscat/train

In [ ]:
# !unzip ../data/dogvscat/test.zip -d ../data/dogvscat/test

In [8]:
!ls ../data/dogvscat/

test  test.zip	train  train.zip


In [9]:
import os
import zipfile
import glob
import time
import pickle
import pprint
import math

%matplotlib inline
import matplotlib.pyplot as plt
import PIL

import numpy as np
import pandas as pd
import sklearn.model_selection

import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torch.utils.data
from torch.utils.data import Dataset, DataLoader

import tqdm
import random
import torchvision.transforms as T

from sklearn.model_selection import train_test_split

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
print(device)

cuda


##Prepare Dataset

In [11]:
TRAIN_DIR = '../data/dogvscat/train/train'
TEST_DIR = '../data/dogvscat/test/test'
train_images = glob.glob(TRAIN_DIR+"/**/**.jpg")
test_images = glob.glob(TEST_DIR+"/**.jpg")

In [12]:
train_images[:5]

['../data/dogvscat/train/train/cats/cat.9769.jpg',
 '../data/dogvscat/train/train/cats/cat.9175.jpg',
 '../data/dogvscat/train/train/cats/cat.9741.jpg',
 '../data/dogvscat/train/train/cats/cat.9358.jpg',
 '../data/dogvscat/train/train/cats/cat.8912.jpg']

In [13]:
dogs_list = [img for img in train_images if img.split("/")[-2] == "dogs"]
cats_list = [img for img in train_images if img.split("/")[-2] == "cats"]

print("Dogs Images: ",len(dogs_list))
print("Cats Images: ",len(cats_list))

class_to_int = {"dogs" : 0, "cats" : 1}
int_to_class = {0 : "dogs", 1 : "cats"}

Dogs Images:  0
Cats Images:  10902


9) Thử nghiệm các phương pháp Augmentation như Flip, Rotation, Random Crop, Bright Contrast để cải thiện kết quả mô hình.

In [14]:
def get_train_transform():
    return T.Compose([
        T.RandomHorizontalFlip(p=0.5), # Random flip with probability = 0.5
        T.RandomRotation(15), # Random rotation with angle <= 15
        # T.ColorJitter(brightness=.5, hue=.3), # Bright contrast
        T.Resize((256, 256)),
        T.RandomResizedCrop(224), # Random crop Image with shape 224
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)), # Normalize according to ImageNet distribution
    ])
    
def get_val_transform():
    return T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

In [15]:
from PIL import Image
class CatDogDataset(Dataset):
    def __init__(self, imgs, class_to_int, mode = "train", 
                 transforms = None):
        super().__init__()
        self.imgs = imgs
        self.class_to_int = class_to_int
        self.mode = mode
        self.transforms = transforms
    def __getitem__(self, idx):
        image_name = self.imgs[idx]
        if self.mode == "train" or self.mode == "val":
            img = Image.open(image_name)
            # img = img.resize((256, 256))
            ### Preparing class label
            label = self.class_to_int[image_name.split("/")[-2]]
            label = torch.tensor(label, dtype = torch.float32)
            ### Apply Transforms on image
            img = self.transforms(img)
            return img, label
        elif self.mode == "test":
            img = Image.open(image_name)
            # img = img.resize((256, 256))
            ### Apply Transforms on image
            img = self.transforms(img)
            return img, image_name
    def __len__(self):
        return len(self.imgs)

In [16]:
train_imgs, val_imgs = train_test_split(train_images, test_size = 0.2)

In [ ]:
train_dataset = CatDogDataset(train_imgs, class_to_int, mode = "train", 
                              transforms = get_train_transform())
val_dataset = CatDogDataset(val_imgs, class_to_int, mode = "val", 
                            transforms = get_val_transform())
test_dataset = CatDogDataset(test_images, class_to_int, mode = "test", 
                             transforms = get_val_transform())

train_data_loader = DataLoader(
    dataset = train_dataset,
    num_workers = 2,
    batch_size = 32,
    shuffle = True
)

val_data_loader = DataLoader(
    dataset = val_dataset,
    num_workers = 2,
    batch_size = 16,
    shuffle = True
)

test_data_loader = DataLoader(
    dataset = test_dataset,
    num_workers = 2,
    batch_size = 1,
    shuffle = False
)

## Train and eval function

In [ ]:
def accuracy(preds, trues):
    preds = [1 if preds[i] >= 0.5 else 0 for i in range(len(preds))]
    acc = [1 if preds[i] == trues[i] else 0 for i in range(len(preds))]
    acc = np.sum(acc) / len(preds)
    return (acc * 100)

In [ ]:
def train_one_epoch(train_data_loader, model, optimizer):
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    # model.to(device)
    model.train()
    
    for images, labels in train_data_loader:
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0], 1)) # [N, 1] - to match with preds shape
        
        #Reseting Gradients
        optimizer.zero_grad()
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = accuracy(preds, labels)
        epoch_acc.append(acc)
        
        #Backward
        _loss.backward()
        optimizer.step()
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    train_logs["loss"].append(epoch_loss)
    train_logs["accuracy"].append(epoch_acc)
    train_logs["time"].append(total_time)
        
    return epoch_loss, epoch_acc, total_time

In [ ]:
def val_one_epoch(val_data_loader, model, best_val_acc, model_name):
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    # model.to(device)
    model.eval()
    
    for images, labels in val_data_loader:
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0], 1)) # [N, 1] - to match with preds shape
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = accuracy(preds, labels)
        epoch_acc.append(acc)
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    val_logs["loss"].append(epoch_loss)
    val_logs["accuracy"].append(epoch_acc)
    val_logs["time"].append(total_time)
    
    ###Saving best model
    if epoch_acc > best_val_acc:
        best_val_acc = epoch_acc
        torch.save(model.state_dict(),model_name+"_best.pth")
        
    return epoch_loss, epoch_acc, total_time, best_val_acc

##Build my model

In [ ]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn,self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=7, padding=0, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(128,256, kernel_size=5, padding=0, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
            )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(256,512, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(2048,1000)
        self.dropout = nn.Dropout()
        self.fc2 = nn.Linear(1000,1)
        self.relu = nn.ReLU()
        self.sigmoid=nn.Sigmoid()
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

To check output shape and number of parameters you have to use `summary` function of `torchsummary` package

In [ ]:
from torchvision import models
from torchsummary import summary

model=cnn()
model.to(device)
summary(model, (3, 224, 224), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 109, 109]          18,944
       BatchNorm2d-2        [-1, 128, 109, 109]             256
              ReLU-3        [-1, 128, 109, 109]               0
         MaxPool2d-4          [-1, 128, 54, 54]               0
            Conv2d-5          [-1, 256, 25, 25]         819,456
       BatchNorm2d-6          [-1, 256, 25, 25]             512
              ReLU-7          [-1, 256, 25, 25]               0
         MaxPool2d-8          [-1, 256, 12, 12]               0
            Conv2d-9            [-1, 512, 5, 5]       1,180,160
      BatchNorm2d-10            [-1, 512, 5, 5]           1,024
             ReLU-11            [-1, 512, 5, 5]               0
        MaxPool2d-12            [-1, 512, 2, 2]               0
           Linear-13                 [-1, 1000]       2,049,000
             ReLU-14                 [-

In [ ]:
print(model)

cnn(
  (layer1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(7, 7), stride=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2048, out_features=1000, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(i

In [ ]:
# Reset gradient
model.zero_grad()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.33)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# Loading model to device
model.to(device)

# No of epochs 
epochs = 15

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    
    ###Training
    loss, acc, _time = train_one_epoch(train_data_loader, model, optimizer)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, model, best_val_acc, "mymodel")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    

7) Grid Search kiến trúc CNN dựa trên việc tổng quát hóa một thiết kế câu 6.

Xem tại [Buổi 4. Deep Neural Network](https://drive.google.com/file/d/1j6eR_g2BXFn2Ml2cpsIQq-sgkn4nV1TL/view?usp=sharing)

In [ ]:
!pip install optuna

In [ ]:
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary

In [ ]:
# def define_model(trial):
#     # We optimize the number of layers, hidden units and dropout ratio in each layer.
#     class CNN_NAS(nn.Module):
#       def __init__(self):
#           super(CNN_NAS,self).__init__()
#           n_layers = trial.suggest_int("n_layers", 2, 5)
#           self.layers = []
#           act_layers = {
#               'relu': nn.ReLU(),
#               'sigmoid': nn.Sigmoid(),
#               'gelu': nn.GELU(),
#           }
#           # in_features = len(features)
#           in_chan = trial.suggest_int("depth", 20, 100)
#           pre_kernel_size = 7
#           x = torch.randn(2, 3, 224, 224) # [b, c, w, h]
#           first_layer = True
#           w = 224
#           for i in range(n_layers):
#             # create search for each cnn block
#             scale_chan = trial.suggest_float("scale", 1.1, 2.5)
#             out_chan = int(in_chan*scale_chan)
#             scale_kernel_size = trial.suggest_float("scale_kernel_size", 1, 2)
#             kernel_size = 2*int((pre_kernel_size-1)/(2*scale_kernel_size))+1
#             stride = trial.suggest_int("stride", 1, 2)
#             act_type = trial.suggest_categorical("act_type", ['relu', 'sigmoid', 'gelu'])
#             p = trial.suggest_float('p', 0, 0.2)
#             if first_layer:
#               in_chan = 3
#               kernel_size = pre_kernel_size
#             if w >= 2:
#               layer = nn.Sequential(
#                 nn.Conv2d(in_chan, out_chan, kernel_size=kernel_size, padding=0, stride=stride),
#                 nn.BatchNorm2d(out_chan),
#                 act_layers[act_type],
#                 nn.MaxPool2d(2),
#                 nn.Dropout(p)
#               )
#               self.layers.append(layer)
#               first_layer = False
#               in_chan = out_chan
#               pre_kernel_size = kernel_size
#               x = layer(x)
#               dim = x.shape
#               w = dim[3]
#           in_features = dim[1]*dim[2]*dim[3]
#           self.fc1 = nn.Linear(in_features,2048)
#           self.dropout = nn.Dropout()
#           self.fc2 = nn.Linear(2048, 1)
#           self.relu = nn.ReLU()
#           self.sigmoid=nn.Sigmoid()
          
#       def forward(self,x):
#           for layer in self.layers:
#             x = layer(x)
#           print('cnn output: ', x.shape)
#           x = x.view(x.size(0),-1)
#           print('x flatten: ', x.shape)
#           x = self.relu(self.fc1(x))
#           x = self.fc2(x)
#           print('x fc2: ', x.shape)
#           x = self.sigmoid(x)
#           return x
#     return CNN_NAS()

#     # in_features = dim[1]*dim[2]*dim[3]
#     # print('dim: ', dim)
#     # print('in_features: ', in_features)
#     # layers.append(nn.Flatten())
#     # layers.append(nn.Linear(in_features, 1000))
#     # layers.append(nn.Dropout(0.5))
#     # layers.append(nn.Linear(1000, 1))
#     # layers.append(nn.ReLU())
#     # layers.append(nn.Sigmoid())
#     # return nn.Sequential(*layers)

In [ ]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 3, 5)
    layers = []
    act_layers = {
        'relu': nn.ReLU(),
        'sigmoid': nn.Sigmoid(),
        'gelu': nn.GELU(),
    }
    # in_features = len(features)
    in_chan = trial.suggest_int("depth", 20, 100)
    pre_kernel_size = 7
    x = torch.randn(2, 3, 224, 224) # [b, c, w, h]
    first_layer = True
    w = 224
    for i in range(n_layers):
      # create search for each CNN block
      scale_chan = trial.suggest_float("scale", 1.1, 2.5)
      out_chan = int(in_chan*scale_chan)
      scale_kernel_size = trial.suggest_float("scale_kernel_size", 1, 2)
      kernel_size = 2*int((pre_kernel_size-1)/(2*scale_kernel_size))+1
      stride = trial.suggest_int("stride", 1, 2)
      act_type = trial.suggest_categorical("act_type", ['relu', 'sigmoid', 'gelu'])
      p = trial.suggest_float('p', 0, 0.2)
      if first_layer:
        in_chan = 3
        kernel_size = pre_kernel_size
      if w >= 2:
        layer = nn.Sequential(
          nn.Conv2d(in_chan, out_chan, kernel_size=kernel_size, padding=0, stride=stride),
          nn.BatchNorm2d(out_chan),
          act_layers[act_type],
          nn.MaxPool2d(2),
          nn.Dropout(p)
        )
        layers.append(layer)
        first_layer = False
        in_chan = out_chan
        pre_kernel_size = kernel_size
        x = layer(x)
        dim = x.shape
        w = dim[3]    
    in_features = dim[1]*dim[2]*dim[3]
    layers.append(nn.Flatten())
    layers.append(nn.Linear(in_features, 1000))
    layers.append(nn.Dropout(0.5))
    layers.append(nn.Linear(1000, 1))
    layers.append(nn.ReLU())
    layers.append(nn.Sigmoid())
    return nn.Sequential(*layers)

In [ ]:
epochs = 15
def objective(trial):

    # Generate the model.
    model = define_model(trial)
    model.to(device)
    print(model)

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    # Learning Rate Scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.33)

    #Loss Function
    criterion = nn.BCELoss()

    best_val_acc = 0
    for epoch in range(epochs):
        
        ###Training
        loss, acc, _time = train_one_epoch(train_data_loader, model, optimizer)
        
        #Print Epoch Details
        print("\nTraining")
        print("Epoch {}".format(epoch+1))
        print("Loss : {}".format(round(loss, 4)))
        print("Acc : {}".format(round(acc, 4)))
        print("Time : {}".format(round(_time, 4)))
        
        ###Validation
        loss, val_acc, _time, best_val_acc = val_one_epoch(val_data_loader, model, best_val_acc, "mymodel")
        
        #Print Epoch Details
        print("\nValidating")
        print("Epoch {}".format(epoch+1))
        print("Loss : {}".format(round(loss, 4)))
        print("Acc : {}".format(round(acc, 4)))
        print("Time : {}".format(round(_time, 4)))
        
        trial.report(val_acc, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_acc

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=1000)

[I 2022-02-27 06:38:15,449] A new study created in memory with name: no-name-7c270ca4-725b-4cf7-a12b-7a050016e6ae


dim:  torch.Size([2, 82, 13, 13])
in_features:  13858
Sequential(
  (0): Sequential(
    (0): Conv2d(3, 47, kernel_size=(7, 7), stride=(2, 2))
    (1): BatchNorm2d(47, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.19391328367873598, inplace=False)
  )
  (1): Sequential(
    (0): Conv2d(47, 82, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(82, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.19391328367873598, inplace=False)
  )
  (2): Flatten(start_dim=1, end_dim=-1)
  (3): Linear(in_features=13858, out_features=1000, bias=True)
  (4): Dropout(p=0.5, inplace=False)
  (5): Linear(in_features=1000, out_features=1, bias=True)
  (6): ReLU()
  (7): Sigmoid()
)

Training
Epoch 1
Loss : 0.0023
Acc : 100.0
Time : 

KeyboardInterrupt: ignored

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
from torchvision import models
from torchsummary import summary

model = define_model(trial).to(device)
summary(model, (3, 224, 224), device=device)

In [ ]:
model

Sequential(
  (0): Linear(in_features=68750, out_features=2048, bias=True)
  (1): Dropout(p=0.5, inplace=False)
  (2): Linear(in_features=2048, out_features=1, bias=True)
  (3): ReLU()
  (4): Sigmoid()
)

##Restnet50

In [ ]:
from torchvision.models import resnet50
model = resnet50(pretrained = True)

# Modifying Head - classifier

model.fc = nn.Sequential(
    nn.Linear(2048, 1, bias = True),
    nn.Sigmoid()
)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

NameError: ignored

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# Loading model to device
model.to(device)

# No of epochs 
epochs = 5

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    
    ###Training
    loss, acc, _time = train_one_epoch(train_data_loader, model, optimizer)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, model, best_val_acc, "resnet50")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Training
Epoch 1
Loss : 0.1442
Acc : 93.905
Time : 550.3182

Validating
Epoch 1
Loss : 0.0447
Acc : 98.4674
Time : 46.7776

Training
Epoch 2
Loss : 0.1121
Acc : 95.375
Time : 549.4432

Validating
Epoch 2
Loss : 0.0438
Acc : 98.1489
Time : 46.1721

Training
Epoch 3
Loss : 0.1136
Acc : 95.24
Time : 549.9613

Validating
Epoch 3
Loss : 0.0392
Acc : 98.6664
Time : 46.1564

Training
Epoch 4
Loss : 0.0999
Acc : 95.81
Time : 548.7609

Validating
Epoch 4
Loss : 0.0395
Acc : 98.3479
Time : 45.8418

Training
Epoch 5
Loss : 0.0981
Acc : 95.795
Time : 548.4862

Validating
Epoch 5
Loss : 0.0565
Acc : 97.8105
Time : 45.8208


##Mobilenet

In [ ]:
from torchvision.models import mobilenet_v2
model=mobilenet_v2(pretrained=True)
model.classifier._modules['1'] = nn.Sequential(
    nn.Linear(1280, 1, bias = True),
    nn.Sigmoid()
)

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# Loading model to device
model.to(device)

# No of epochs 
epochs = 5

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    
    ###Training
    loss, acc, _time = train_one_epoch(train_data_loader, model, optimizer)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, model, best_val_acc, "mobilenet_v2")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Training
Epoch 1
Loss : 0.1475
Acc : 93.85
Time : 211.3811

Validating
Epoch 1
Loss : 0.0463
Acc : 98.6465
Time : 40.5227

Training
Epoch 2
Loss : 0.1124
Acc : 95.215
Time : 209.3682

Validating
Epoch 2
Loss : 0.0418
Acc : 98.5669
Time : 45.9461

Training
Epoch 3
Loss : 0.1017
Acc : 95.675
Time : 208.8129

Validating
Epoch 3
Loss : 0.0378
Acc : 98.6266
Time : 40.8667

Training
Epoch 4
Loss : 0.095
Acc : 95.99
Time : 209.7385

Validating
Epoch 4
Loss : 0.0446
Acc : 98.5271
Time : 45.0689

Training
Epoch 5
Loss : 0.0906
Acc : 96.275
Time : 213.2981

Validating
Epoch 5
Loss : 0.0461
Acc : 98.3479
Time : 40.9497


##Predict

In [ ]:
model_file="resnet50_best.pth"
model_name=model_file.split(".")[0]
# model=mobilenet_v2(pretrained=False)
# model.classifier._modules['1'] = nn.Sequential(
#     nn.Linear(1280, 1, bias = True),
#     nn.Sigmoid()
# )
model = resnet50(pretrained = True)
model.fc = nn.Sequential(
    nn.Linear(2048, 1, bias = True),
    nn.Sigmoid()
)
model.load_state_dict(torch.load(model_file))
model.to(device)
model.eval()  # ensure we're in eval mode

test_predictions = []
test_idx = []
print('Testing...')
for X,id_ in test_data_loader:
    with torch.no_grad():
        X = X.to(device)
        predictions = model(X)
        test_idx.extend(list(id_))
        test_predictions.extend([1 if predictions[i] < 0.5 else 0 for i in range(len(predictions))])  

submission = pd.DataFrame({'id': test_idx, 'labels': test_predictions}).sort_values(by='id')
submission.to_csv(f'submission_'+model_name+'.csv', index=False)
print(f'Submission saved')

Testing...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Submission saved


## Result


*   My mode: 0.81853. 
*   Mobilenet: 0.98666
*   Resnet: 0.98958



##Augmentation

In [ ]:
def get_train_transform_new():
    return T.Compose([
        T.RandomHorizontalFlip(p=0.5),
        T.RandomVerticalFlip(p=0.2),
        T.RandomAutocontrast(p=0.2),
        T.ColorJitter(brightness=0.2,
            contrast=0.2,
            saturation=0.2),
        T.RandomPerspective(p=0.2),
        T.RandomRotation(15),
        T.Resize((256, 256)),
        T.RandomResizedCrop(224),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])
    
def get_val_transform():
    return T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

###Example with mobilenet

In [ ]:
train_dataset = CatDogDataset(train_imgs, class_to_int, mode = "train", transforms = get_train_transform_new())
train_data_loader = DataLoader(
    dataset = train_dataset,
    num_workers = 4,
    batch_size = 32,
    shuffle = True
)
from torchvision.models import mobilenet_v2
model=mobilenet_v2(pretrained=True)
model.classifier._modules['1'] = nn.Sequential(
    nn.Linear(1280, 1, bias = True),
    nn.Sigmoid()
)
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)
#Loss Function
criterion = nn.BCELoss()
# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}
# Loading model to device
model.to(device)
# No of epochs 
epochs = 5
best_val_acc = 0
for epoch in range(epochs):
    ###Training
    loss, acc, _time = train_one_epoch(train_data_loader)
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, best_val_acc, "mobilenet_v2_augment")
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    